<a href="https://colab.research.google.com/github/MeenalShah13/SentimentAnalysisCOVID/blob/main/Sentiment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np


In [14]:
tweets=pd.read_csv('covid19_tweets.csv')
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [16]:
df = tweets

In [17]:
df.shape

(179108, 13)

In [18]:
df.describe()

,user_followers,user_friends,user_favourites
count,1.791080e+05,179108.000000,1.791080e+05
mean,1.090555e+05,2121.701566,1.444411e+04
std,8.414670e+05,9162.553072,4.452270e+04
min,0.000000e+00,0.000000,0.000000e+00
25%,1.720000e+02,148.000000,2.060000e+02
50%,9.920000e+02,542.000000,1.791000e+03
75%,5.284000e+03,1725.250000,9.388000e+03
max,4.944256e+07,497363.000000,2.047197e+06


In [19]:
df.isnull().sum() 

user_name               0
user_location       36771
user_description    10286
user_created            0
user_followers          0
user_friends            0
user_favourites         0
user_verified           0
date                    0
text                    0
hashtags            51334
source                 77
is_retweet              0
dtype: int64

In [20]:
df = df.dropna()

In [21]:
df.duplicated().sum()

0

In [24]:
df = df.drop_duplicates(subset=None, keep='first', inplace=False)

In [25]:
df.isnull().sum()

user_name           0
user_location       0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
dtype: int64

In [26]:
df.shape

(99138, 13)

In [27]:
sample = df.sample(n=15000)
sample.to_csv('sample_data.csv',index=False)

In [28]:
sample.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
63854,Gautam Aggarwal,"New Delhi, India",Businessman/ BJP & RSS Member / Khand Pramukh ...,2014-05-24 05:18:54,1958,810,94963,False,2020-08-02 18:46:05,Prayers for your good health and speedy recove...,['COVID19'],Twitter for Android,False
163718,NDI Parties Team,"Washington, D.C.",@NDI's Political Parties Team seeks to foster ...,2013-11-14 19:16:32,3306,759,958,False,2020-08-22 02:59:00,"Around the world, @NDI is helping legislatures...",['PoliticalParties'],TweetDeck,False
118424,Emirates News,"Dubai, UAE",#EmiratesNews the most balanced and credible s...,2019-10-24 15:46:15,1916,11,74,True,2020-08-13 10:21:32,"#UAE Health Ministry conducts 68,964 additiona...","['UAE', 'COVID19']",Hootsuite Inc.,False
38896,Tvasta,"Chennai, India",We are an emerging player in the #Additive Man...,2018-09-26 10:24:52,75,62,75,False,2020-07-28 03:21:30,A Possible Weapon Against the Pandemic: Printi...,"['bioprinting', '3dprinting']",Twitter Web App,False
162080,Michael A. Freeman,Int'l Criminal Court The Hague,I'm an FDR-style American Democratic Socialist...,2019-08-10 23:27:39,234,745,15248,False,2020-08-22 04:38:09,"@GOP @VP Pence, u already abdicated daily #Cov...",['CovidTaskForce'],Twitter Web App,False


In [30]:
df1 = sample.drop(['user_created','user_description','user_followers','user_friends','user_favourites','user_verified','source','is_retweet'],axis=1)
df1.head()

,user_name,user_location,date,text,hashtags
63854,Gautam Aggarwal,"New Delhi, India",2020-08-02 18:46:05,Prayers for your good health and speedy recove...,['COVID19']
163718,NDI Parties Team,"Washington, D.C.",2020-08-22 02:59:00,"Around the world, @NDI is helping legislatures...",['PoliticalParties']
118424,Emirates News,"Dubai, UAE",2020-08-13 10:21:32,"#UAE Health Ministry conducts 68,964 additiona...","['UAE', 'COVID19']"
38896,Tvasta,"Chennai, India",2020-07-28 03:21:30,A Possible Weapon Against the Pandemic: Printi...,"['bioprinting', '3dprinting']"
162080,Michael A. Freeman,Int'l Criminal Court The Hague,2020-08-22 04:38:09,"@GOP @VP Pence, u already abdicated daily #Cov...",['CovidTaskForce']


In [31]:
df1['user_location'].value_counts()

India                          442
United States                  254
New Delhi, India               206
Mumbai, India                  187
Washington, DC                 125
                              ... 
Mississippi                      1
Sioux Falls, SD                  1
Nairobi Metropolitan, Kenya      1
Newfangled                       1
Yorkshire Leeds UK               1
Name: user_location, Length: 5555, dtype: int64

In [32]:
df1.shape

(15000, 5)

In [33]:
df1.to_csv('sampledata.csv',index=False)

ANALYSIS


In [46]:
df1 = pd.read_csv("sampledata.csv")

In [38]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [39]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [50]:
blanks = []  # start with an empty list

for i,un,ul,d,t,h in df1.itertuples():  # iterate over the DataFrame
    if type(t)==str:            # avoid NaN values
        if t.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df1.drop(blanks, inplace=True)
df1.shape

(15000, 5)

In [57]:
df1['scores'] = df1['text'].apply(lambda review: sid.polarity_scores(review))

df1['compound']  = df1['scores'].apply(lambda score_dict: score_dict['compound'])


df1['label'] = df1['compound'].apply(lambda c: 'pos' if c >0 else ('neu' if c==0 else 'neg'))


df1.head()

,user_name,user_location,date,text,hashtags,scores,compound,label
0,Gautam Aggarwal,"New Delhi, India",2020-08-02 18:46:05,Prayers for your good health and speedy recove...,['COVID19'],"{'neg': 0.0, 'neu': 0.722, 'pos': 0.278, 'comp...",0.6124,pos
1,NDI Parties Team,"Washington, D.C.",2020-08-22 02:59:00,"Around the world, @NDI is helping legislatures...",['PoliticalParties'],"{'neg': 0.0, 'neu': 0.71, 'pos': 0.29, 'compou...",0.6486,pos
2,Emirates News,"Dubai, UAE",2020-08-13 10:21:32,"#UAE Health Ministry conducts 68,964 additiona...","['UAE', 'COVID19']","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu
3,Tvasta,"Chennai, India",2020-07-28 03:21:30,A Possible Weapon Against the Pandemic: Printi...,"['bioprinting', '3dprinting']","{'neg': 0.167, 'neu': 0.833, 'pos': 0.0, 'comp...",-0.2960,neg
4,Michael A. Freeman,Int'l Criminal Court The Hague,2020-08-22 04:38:09,"@GOP @VP Pence, u already abdicated daily #Cov...",['CovidTaskForce'],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neu


In [58]:
df1['label'].value_counts()

neu    5617
pos    5549
neg    3834
Name: label, dtype: int64

In [71]:
import re
contractions_dict = {"can`t":"can not",
                     "won`t":"will not",
                     "don`t":"do not",
                     "aren`t":"are not",
                     "i`d":"i would",
                     "couldn`t": "could not",
                     "shouldn`t": "should not",
                     "wouldn`t": "would not",
                     "isn`t": "is not",
                     "it`s": "it is",
                     "didn`t": "did not",
                     "weren`t": "were not",
                     "mustn`t": "must not",
                    }

def prepare_data(df1:pd.DataFrame) -> pd.DataFrame:
    
    df1["text"] = df1["text"] \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))
        
    df1["label"] = df1["label"].map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )
    return df1.text.values, df1.label.values
def replace_words(string:str, dictionary:dict):
    for k, v in dictionary.items():
        string = string.replace(k, v)
    return string


In [72]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

X = df1.text
Y =df1.label

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



X_train = X_train \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))

X_test = X_test \
              .apply(lambda x: re.split('http:\/\/.*', str(x))[0]) \
              .str.lower() \
              .apply(lambda x: replace_words(x,contractions_dict))

y_train =  y_train.map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )
y_test = y_test.map(
                        {"neu": 1, "neg":0, "pos":2 }
                        )


(12000,) (12000,)
(3000,) (3000,)


In [73]:
train_tweets, train_labels = X_train,y_train
test_tweets, test_labels = X_test,y_test

In [75]:
pip install keras


_______________________________________

In [81]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tweets)
train_tokenized = tokenizer.texts_to_matrix(
                             train_tweets,
                             mode='tfidf'
                             )
test_tokenized = tokenizer.texts_to_matrix(
                             test_tweets,
                             mode='tfidf'
                             )

In [82]:
from  sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(
                                n_estimators=500, 
                                min_samples_leaf=2,
                                oob_score=True,
                                n_jobs=-1,
                                )
forest.fit(train_tokenized,train_labels)
print(f"Train score: {forest.score(train_tokenized,train_labels)}")
print(f"OOB score: {forest.oob_score_}")

Train score: 0.8411666666666666
OOB score: 0.7286666666666667


In [86]:
print("Test score:", forest.score(test_tokenized,test_labels))

Test score: 0.7216666666666667


IMBALANCED


In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [62]:
y = df1.label
x = df1.text

imb = LogisticRegression().fit(x,y)

imb_pred = imb.predict(x)
print(accuracy_score(imb_pred,y))

ValueError: ignored